<a href="https://colab.research.google.com/github/SouparnaChatterjee/Meta-AIML/blob/main/LLP_bot_draft1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages
!pip install nltk ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.0 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import nltk
import re
import random
import json
from nltk.tokenize import word_tokenize
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets

# Download necessary NLTK data
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
class MaliziasignChatbot:
    def __init__(self):
        # Define intents and their patterns
        self.intents = {
            "greeting": ["hello", "hi", "hey", "greetings", "good morning", "good afternoon", "good evening"],
            "services": ["what services", "services offer", "provide", "services available", "what do you do", "services"],
            "location": ["where", "location", "address", "find you", "office", "directions"],
            "portfolio": ["portfolio", "work", "examples", "showcase", "previous work", "projects"],
            "contact": ["contact", "reach", "phone", "email", "call", "whatsapp"],
            "lead": ["interested", "sign up", "more information", "quote", "inquiry", "request quote"],
            "hours": ["hours", "working hours", "open", "business hours", "when are you open", "schedule"],
            "about": ["about", "company", "history", "team", "who are you", "background"]
        }

        # Define responses for each intent
        self.responses = {
            "greeting": "Hello! Welcome to Maliziasign. How can I help you today?",
            "services": "We offer the following services:\n- Graphic Design\n- Web Development\n- Branding\n- Print Services\n- Digital Marketing\n- Signage Solutions",
            "location": "Our office is located at: 123 Design Street, Creative District, Example City. Would you like directions?",
            "portfolio": "Our portfolio includes projects for clients in various industries. Would you like to see specific categories?",
            "contact": "You can reach us through:\n- Phone: (123) 456-7890\n- Email: contact@maliziasign.com\n- WhatsApp: +1 234 567 8901\n- Social Media: @Maliziasign",
            "lead": "We'd be happy to provide a quote for your project. Please provide some details:",
            "hours": "Our working hours are:\nMonday-Friday: 9:00 AM - 6:00 PM\nSaturday: 10:00 AM - 2:00 PM\nSunday: Closed",
            "about": "Maliziasign is a creative design agency established in 2010. We specialize in creating impactful visual communications for businesses of all sizes.",
            "default": "I'm not sure what you're looking for. Could you please be more specific or choose from the options below?"
        }

        # Define options for each intent
        self.options = {
            "greeting": ["What services do you offer?", "View portfolio", "Contact information", "Request a quote", "Working hours", "About us"],
            "services": ["Graphic Design", "Web Development", "Branding", "Print Services", "Digital Marketing", "Signage Solutions", "Request a quote", "Back to main menu"],
            "location": ["Get directions", "Working hours", "Contact us", "Back to main menu"],
            "portfolio": ["Branding Projects", "Web Design", "Print Materials", "Signage Projects", "All Projects", "Back to main menu"],
            "contact": ["Call now", "Send email", "WhatsApp", "Request callback", "Back to main menu"],
            "lead": ["Submit quote request", "Talk to a representative", "View services", "Back to main menu"],
            "hours": ["Contact us", "Get location", "Back to main menu"],
            "about": ["Our team", "Company history", "Services", "Contact us", "Back to main menu"],
            "default": ["Services", "Portfolio", "Contact", "Quote", "Working hours", "About us"]
        }

        # Service details
        self.service_details = {
            "graphic design": "Our Graphic Design services include logo design, marketing materials, social media graphics, and more.",
            "web development": "We create responsive, user-friendly websites optimized for performance and conversion.",
            "branding": "Our Branding services help establish a cohesive visual identity across all touchpoints.",
            "print services": "We offer high-quality printing for business cards, brochures, banners, and more.",
            "digital marketing": "Our Digital Marketing services include social media management, SEO, and online advertising.",
            "signage solutions": "We design and produce indoor/outdoor signage, vehicle wraps, and exhibition displays."
        }

        # Portfolio categories
        self.portfolio_categories = {
            "branding projects": "Our branding portfolio includes complete visual identity systems for clients in retail, hospitality, and technology sectors.",
            "web design": "Our web design portfolio showcases responsive websites, e-commerce platforms, and interactive web applications.",
            "print materials": "Our print portfolio includes brochures, catalogs, packaging designs, and various marketing materials.",
            "signage projects": "Our signage portfolio features storefront signs, wayfinding systems, vehicle wraps, and exhibition booths.",
            "all projects": "View our complete portfolio at maliziasign.com/portfolio or let me know which category interests you most."
        }

        # Chat history
        self.chat_history = []

        # Lead information
        self.lead_info = {"name": "", "email": "", "phone": "", "project_type": "", "details": ""}
        self.collecting_lead = False
        self.lead_field = ""

    def process_input(self, user_input):
        # Check if we're collecting lead information
        if self.collecting_lead:
            return self.collect_lead_info(user_input)

        # Process normal input
        user_input = user_input.lower()

        # Check for service details request
        for service, details in self.service_details.items():
            if service in user_input:
                return {"response": details, "options": self.options["services"]}

        # Check for portfolio category request
        for category, details in self.portfolio_categories.items():
            if category.lower() in user_input:
                return {"response": details, "options": self.options["portfolio"]}

        # Check for intent matches
        for intent, patterns in self.intents.items():
            if any(pattern in user_input for pattern in patterns):
                return {"response": self.responses[intent], "options": self.options[intent]}

        # Default response if no intent is matched
        return {"response": self.responses["default"], "options": self.options["default"]}

    def collect_lead_info(self, user_input):
        if self.lead_field == "name":
            self.lead_info["name"] = user_input
            self.lead_field = "email"
            return {"response": f"Thanks, {user_input}! What's your email address?", "options": []}

        elif self.lead_field == "email":
            self.lead_info["email"] = user_input
            self.lead_field = "phone"
            return {"response": "Great! What's your phone number?", "options": []}

        elif self.lead_field == "phone":
            self.lead_info["phone"] = user_input
            self.lead_field = "project_type"
            return {"response": "What type of project are you interested in?", "options": ["Graphic Design", "Web Development", "Branding", "Print Services", "Digital Marketing", "Signage", "Other"]}

        elif self.lead_field == "project_type":
            self.lead_info["project_type"] = user_input
            self.lead_field = "details"
            return {"response": "Please provide a brief description of your project:", "options": []}

        elif self.lead_field == "details":
            self.lead_info["details"] = user_input
            self.collecting_lead = False
            self.lead_field = ""

            # In a real application, you would save this information to a database
            lead_summary = f"Name: {self.lead_info['name']}\nEmail: {self.lead_info['email']}\nPhone: {self.lead_info['phone']}\nProject Type: {self.lead_info['project_type']}\nDetails: {self.lead_info['details']}"

            return {
                "response": f"Thank you for your inquiry! Our team will review your project details and contact you within 24 hours with a quote.",
                "options": self.options["greeting"]
            }

    def handle_option(self, option):
        # Handle lead form submission
        if option == "Submit quote request":
            self.collecting_lead = True
            self.lead_field = "name"
            return {"response": "Please provide your name:", "options": []}

        # Handle back to main menu
        if option == "Back to main menu":
            return {"response": "What would you like to know about?", "options": self.options["greeting"]}

        # Handle service details
        for service in self.service_details.keys():
            if service.title() == option:
                return {"response": self.service_details[service], "options": self.options["services"]}

        # Handle portfolio categories
        for category in self.portfolio_categories.keys():
            if category.title() == option:
                return {"response": self.portfolio_categories[category], "options": self.options["portfolio"]}

        # Handle other options based on content
        if "services" in option.lower() or "offer" in option.lower():
            return {"response": self.responses["services"], "options": self.options["services"]}
        elif "portfolio" in option.lower() or "projects" in option.lower() or "view" in option.lower():
            return {"response": self.responses["portfolio"], "options": self.options["portfolio"]}
        elif "contact" in option.lower() or "reach" in option.lower():
            return {"response": self.responses["contact"], "options": self.options["contact"]}
        elif "quote" in option.lower() or "request" in option.lower():
            return {"response": self.responses["lead"], "options": self.options["lead"]}
        elif "hours" in option.lower() or "time" in option.lower():
            return {"response": self.responses["hours"], "options": self.options["hours"]}
        elif "about" in option.lower() or "company" in option.lower():
            return {"response": self.responses["about"], "options": self.options["about"]}
        elif "location" in option.lower() or "direction" in option.lower():
            return {"response": self.responses["location"], "options": self.options["location"]}
        else:
            return {"response": f"You selected: {option}", "options": self.options["default"]}

In [4]:
def create_chatbot_interface():
    # Create chatbot instance
    chatbot = MaliziasignChatbot()

    # Create widgets
    output = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '10px', 'height': '400px', 'overflow_y': 'auto'})
    text_input = widgets.Text(placeholder='Type your message here...', layout={'width': '80%'})
    send_button = widgets.Button(description='Send', button_style='primary', layout={'width': '18%'})
    input_box = widgets.HBox([text_input, send_button])

    # Options area
    options_output = widgets.Output()

    # Display initial message and options
    with output:
        print("Bot: Hello! Welcome to Maliziasign. How can I help you today?")

    # Display initial options
    initial_options = chatbot.options["greeting"]
    option_buttons = []

    for option in initial_options:
        button = widgets.Button(description=option, layout={'margin': '5px'})
        option_buttons.append(button)

    with options_output:
        display(widgets.HBox(option_buttons))

    # Function to handle user input
    def on_send_button_clicked(b):
        user_message = text_input.value
        if not user_message.strip():
            return

        with output:
            print(f"You: {user_message}")

        # Get response from chatbot
        response_data = chatbot.process_input(user_message)

        with output:
            print(f"Bot: {response_data['response']}")

        # Update options
        update_options(response_data['options'])

        # Clear input
        text_input.value = ''

    # Function to handle option selection
    def on_option_clicked(b):
        option = b.description

        with output:
            print(f"You: {option}")

        # Get response from chatbot
        response_data = chatbot.handle_option(option)

        with output:
            print(f"Bot: {response_data['response']}")

        # Update options
        update_options(response_data['options'])

    # Function to update option buttons
    def update_options(options):
        # Clear previous options
        options_output.clear_output()

        # Create new option buttons
        option_buttons = []
        for option in options:
            button = widgets.Button(description=option, layout={'margin': '5px'})
            button.on_click(on_option_clicked)
            option_buttons.append(button)

        # Display new options
        if option_buttons:
            with options_output:
                # Display buttons in rows of 3
                rows = [option_buttons[i:i+3] for i in range(0, len(option_buttons), 3)]
                for row in rows:
                    display(widgets.HBox(row))

    # Connect event handlers
    send_button.on_click(on_send_button_clicked)
    text_input.on_submit(lambda text_input: on_send_button_clicked(None))

    # Connect initial option buttons
    for button in option_buttons:
        button.on_click(on_option_clicked)

    # Create main layout
    main_layout = widgets.VBox([
        widgets.HTML("<h2 style='text-align:center;background-color:#4a69bd;color:white;padding:10px;border-radius:5px;'>Maliziasign Chatbot</h2>"),
        output,
        input_box,
        widgets.HTML("<p style='margin:5px 0;font-weight:bold;'>Options:</p>"),
        options_output
    ])

    return main_layout

In [5]:
# Create and display the chatbot interface
chatbot_interface = create_chatbot_interface()
display(chatbot_interface)